In [24]:
from scipy.optimize import linprog
import numpy as np

###seeing if git works 

## Matroid Intersection Method for k = 2. 

In [6]:
obj = [0, 0, 0, -1] # beta, tau, gamma, z

lhs_ineq = [[0.0, -5/252, 0.0, 1.0],
            [1/72, -1/21, 0.0, 1.0],
            [0.0, -1/21, 1/36, 1.0],
            [0.0, 1/54, -1/18, 1.0], 
            [0.0, 0.0, -1/27, 1.0],
            [1/54, 0.0, -1/18, 1.0], 
            [-1/72, 0.0, 0.0, 1.0], 
            [-1/24, 5/72, 0.0, 1.0],
            [-1/24, 0.0, 5/72, 1.0], 
            [-1/18, 1/9, 0.0, 1.0], 
            [-1.0, 1.0, 0.0, 0.0], 
            [1.0, 0.0, 0.0, 0.0],
            [-0.5, 1.0, 0.0, 0.0],
            [-0.5, 0.0, 1.0, 0.0],
            [0.0, -1/36, 0.0, 1.0],
            [-1/36, 0.0, 0.0, 1.0]]


rhs_ineq = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1/12, 0.0, 0.0, 0.0, 0.0]
            
bnd = [(0, float("inf")), (0, float("inf")), (0, float("inf")), (0, float("inf"))] 
            
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method="revised simplex")

In [7]:
opt

     con: array([], dtype=float64)
     fun: -0.000578703703703704
 message: 'Optimization terminated successfully.'
     nit: 11
   slack: array([ 1.44675926e-04, -5.42101086e-19,  9.64506173e-05,  8.68055556e-04,
        8.35905350e-04, -4.33680869e-19,  5.78703704e-04,  3.61689815e-04,
        2.41126543e-04,  5.42101086e-19,  4.68750000e-02,  0.00000000e+00,
        5.20833333e-03,  3.47222222e-03,  4.34027778e-04,  1.73611111e-03])
  status: 0
 success: True
       x: array([0.08333333, 0.03645833, 0.03819444, 0.0005787 ])

## Max Entropy Method for k = 2. 

### Uniform p approach

In [8]:
c = [0, 0, 0, -1] # tau, beta, gamma, z

p_sp = 1/36 #128/6561
p = 1/18 #13/144 
p_hs = 1/21 #p #4/27


## CHARGING DEGREE
p_internal, p_external = min(p, p_hs), max(p, p_hs) # NOTE! in non-uniform, we can do p >= 1/9. 
HALLS_CONSTANT = 1/2
P_CUT_ODD_DEGREE = 1
P_CUT_ODD_CYCLE = 1/2

special_edge = ([-1 * p_sp, 0, 0, 1], 0) # Special edge 
degree_degree = ([-1 * (p_internal - p_external * HALLS_CONSTANT*P_CUT_ODD_DEGREE), 0, 0, 1], 0) # Degree edge, all external edges are degree
degree_k5 = ([-1 * p_internal, 0, p_external * HALLS_CONSTANT*P_CUT_ODD_DEGREE, 1],0) # Degree edge, all external edges are K_5
degree_cycle = ([-1 * p_internal, p_external * HALLS_CONSTANT*P_CUT_ODD_CYCLE, 0, 1],0) # Degree edge, all external edges are cycle



## CHARGING K5
p_internal, p_external = p, max(p, p_hs)  # NOTE! in non-uniform, we can do p_internal >= 1/9. 
HALLS_CONSTANT = 2/3
P_CUT_ODD_K5 = 1/2

k5_degree = ([p_external*HALLS_CONSTANT*P_CUT_ODD_K5, 0, -1 * p_internal, 1], 0)
k5_k5 = ([0, 0, -1 * p_internal + p_external*HALLS_CONSTANT*P_CUT_ODD_K5, 1], 0)
k5_cycle = ([0, p_external*HALLS_CONSTANT*P_CUT_ODD_CYCLE, -1 * p_internal, 1], 0)



## CHARGING CYCLE
gamma_le_betaover2 = ([0, -1/2, 1, 0], 0)
tau_le_betaover2 = ([1, -1/2, 0, 0], 0)

cycle_i = ([0, -1 * p * 1/4, 0, 1],0) # Constraint from case 7.1.2 (i)
cycle_ii = ([p * 5/4, -1 * p * 3/4, 0 , 1],0) # Constraint from case 7.1.2 (ii)
cycle_iii = ([0, -1 * p * 3/4, p * 5/4 , 1],0) # Constraint from case 7.1.2 (iii)
cycle_iv = ([2 * p, -1 * p, 0, 1],0) # Constraint from case 7.1.2 (iv)

### 

constraints = [special_edge, degree_degree, degree_k5, degree_cycle, k5_degree, k5_k5, k5_cycle, gamma_le_betaover2, tau_le_betaover2, 
              cycle_i, cycle_ii, cycle_iii, cycle_iv]


lhs_ineq_1 = np.array([cons[0] for cons in constraints])
rhs_ineq_1 = np.array([cons[1] for cons in constraints])

print(lhs_ineq_1)

bnd = [(0, float("inf")) for variable in c] 

opt_1 = linprog(c=c, A_ub=lhs_ineq_1, b_ub=rhs_ineq_1, A_eq=[[0, 1, 0, 0]], b_eq=[1/12], bounds=bnd, method="revised simplex")

[[-0.02777778  0.          0.          1.        ]
 [-0.01984127  0.          0.          1.        ]
 [-0.04761905  0.          0.02777778  1.        ]
 [-0.04761905  0.01388889  0.          1.        ]
 [ 0.01851852  0.         -0.05555556  1.        ]
 [ 0.          0.         -0.03703704  1.        ]
 [ 0.          0.01851852 -0.05555556  1.        ]
 [ 0.         -0.5         1.          0.        ]
 [ 1.         -0.5         0.          0.        ]
 [ 0.         -0.01388889  0.          1.        ]
 [ 0.06944444 -0.04166667  0.          1.        ]
 [ 0.         -0.04166667  0.06944444  1.        ]
 [ 0.11111111 -0.05555556  0.          1.        ]]


In [9]:
opt_1

     con: array([0.])
     fun: -0.0005787037037037039
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 4.34027778e-04,  1.44675926e-04,  9.64506173e-05,  0.00000000e+00,
        8.68055556e-04,  8.35905350e-04, -2.16840434e-19,  3.47222222e-03,
        5.20833333e-03,  5.78703704e-04,  3.61689815e-04,  2.41126543e-04,
       -2.16840434e-19])
  status: 0
 success: True
       x: array([0.03645833, 0.08333333, 0.03819444, 0.0005787 ])

### Choose a sampler with probability lambda

In [34]:
c = [0, 0, 0, -1] # tau, beta, gamma, z

lam = 0
values = []
while lam <= 1:
    p_sp = lam * 1/36 + (1 -lam) * 128/6561
    p = lam * 1/18 + (1 - lam) * 13/144 
    p_hs = min(lam * 1/21 + (1 - lam) * 4/27, p)


    ## CHARGING DEGREE
    p_internal, p_external = min(p, p_hs), max(p, p_hs) # NOTE! in non-uniform, we can do p >= 1/9. 
    HALLS_CONSTANT = 1/2
    P_CUT_ODD_DEGREE = 1
    P_CUT_ODD_CYCLE = 1/2

    special_edge = ([-1 * p_sp, 0, 0, 1], 0) # Special edge 
    degree_degree = ([-1 * (p_internal - p_external * HALLS_CONSTANT*P_CUT_ODD_DEGREE), 0, 0, 1], 0) # Degree edge, all external edges are degree
    degree_k5 = ([-1 * p_internal, 0, p_external * HALLS_CONSTANT*P_CUT_ODD_DEGREE, 1],0) # Degree edge, all external edges are K_5
    degree_cycle = ([-1 * p_internal, p_external * HALLS_CONSTANT*P_CUT_ODD_CYCLE, 0, 1],0) # Degree edge, all external edges are cycle



    ## CHARGING K5
    p_internal, p_external = p, max(p, p_hs)  # NOTE! in non-uniform, we can do p_internal >= 1/9. 
    HALLS_CONSTANT = 2/3
    P_CUT_ODD_K5 = 1/2

    k5_degree = ([p_external*HALLS_CONSTANT*P_CUT_ODD_K5, 0, -1 * p_internal, 1], 0)
    k5_k5 = ([0, 0, -1 * p_internal + p_external*HALLS_CONSTANT*P_CUT_ODD_K5, 1], 0)
    k5_cycle = ([0, p_external*HALLS_CONSTANT*P_CUT_ODD_CYCLE, -1 * p_internal, 1], 0)



    ## CHARGING CYCLE
    gamma_le_betaover2 = ([0, -1/2, 1, 0], 0)
    tau_le_betaover2 = ([1, -1/2, 0, 0], 0)

    cycle_i = ([0, -1 * p * 1/4, 0, 1],0) # Constraint from case 7.1.2 (i)
    cycle_ii = ([p * 5/4, -1 * p * 3/4, 0 , 1],0) # Constraint from case 7.1.2 (ii)
    cycle_iii = ([0, -1 * p * 3/4, p * 5/4 , 1],0) # Constraint from case 7.1.2 (iii)
    cycle_iv = ([2 * p, -1 * p, 0, 1],0) # Constraint from case 7.1.2 (iv)

    ### 

    constraints = [special_edge, degree_degree, degree_k5, degree_cycle, k5_degree, k5_k5, k5_cycle, gamma_le_betaover2, tau_le_betaover2, 
                  cycle_i, cycle_ii, cycle_iii, cycle_iv]


    lhs_ineq_1 = np.array([cons[0] for cons in constraints])
    rhs_ineq_1 = np.array([cons[1] for cons in constraints])

    #print(lhs_ineq_1)

    bnd = [(0, float("inf")) for variable in c] 

    opt_1 = linprog(c=c, A_ub=lhs_ineq_1, b_ub=rhs_ineq_1, A_eq=[[0, 1, 0, 0]], b_eq=[1/12], bounds=bnd, method="revised simplex")
    values.append(-1 * opt_1.fun)
    lam += 0.0001
print(max(values))

0.0008639831961591429


In [32]:
opt_1.fun

-0.00073361619880999

## Heather: Max Entropy Solely

In [27]:
c = [0, 0, 0, -1] # tau, beta, gamma, z

p_sp = 128/6561
p = 13/144 




## CHARGING DEGREE
#p_internal, p_external = min(p, p_hs), max(p, p_hs) # NOTE! in non-uniform, we can do p >= 1/9. 
HALLS_CONSTANT = 1/2
P_CUT_ODD_DEGREE = 1
P_CUT_ODD_CYCLE = 1/2

special_edge = ([-1 * p_sp, 0, 0, 1], 0) # Special edge 
degree_degree = ([-1 * (p - p * HALLS_CONSTANT*P_CUT_ODD_DEGREE), 0, 0, 1], 0) # Degree edge, all external edges are degree
degree_k5 = ([-1 * p, 0, p * HALLS_CONSTANT*P_CUT_ODD_DEGREE, 1],0) # Degree edge, all external edges are K_5
degree_cycle = ([-1 * p, p * HALLS_CONSTANT*P_CUT_ODD_CYCLE, 0, 1],0) # Degree edge, all external edges are cycle



## CHARGING K5
#p_internal, p_external = p, max(p, p_hs)  # NOTE! in non-uniform, we can do p_internal >= 1/9. 
HALLS_WEAK_CONSTANT = 2/3
P_CUT_ODD_K5 = 1/2

k5_degree = ([p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, 0, -1 * p, 1], 0)
k5_k5 = ([0, 0, -1 * p + p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, 1], 0)
k5_cycle = ([0, p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, -1 * p, 1], 0)



## CHARGING CYCLE (added in one more case here)
gamma_le_betaover2 = ([0, -1/2, 1, 0], 0) #assume cycle edges give worst case (see 7.1.2 (iv))
tau_le_betaover2 = ([1, -1/2, 0, 0], 0) #assume cycle edges give worst case (see 7.1.2 (iv))

cycle_i = ([0, -1 * p * 1/4, 0, 1],0) # Constraint from case 7.1.2 (i)
cycle_ii = ([p * 5/4, -1 * p * 3/4, 0 , 1],0) # Constraint from case 7.1.2 (ii), degree case
cycle_ii_2 = ([0, -1 * p * 3/4, p * 5/4 , 1],0) # Constraint from case 7.1.2 (ii), K5 case
cycle_iii = ([0, -1 * p * 1/2, 0, 1],0) # Constraint from case 7.1.2 (iii)
cycle_iv = ([2 * p, -1 * p, 0, 1],0) # Constraint from case 7.1.2 (iv)

### 

constraints = [special_edge, degree_degree, degree_k5, degree_cycle, k5_degree, k5_k5, k5_cycle, gamma_le_betaover2, tau_le_betaover2, 
              cycle_i, cycle_ii, cycle_ii_2, cycle_iii, cycle_iv]


lhs_ineq_1 = np.array([cons[0] for cons in constraints])
rhs_ineq_1 = np.array([cons[1] for cons in constraints])

print(lhs_ineq_1)

bnd = [(0, float("inf")) for variable in c] 

opt_1 = linprog(c=c, A_ub=lhs_ineq_1, b_ub=rhs_ineq_1, A_eq=[[0, 1, 0, 0]], b_eq=[1/12], bounds=bnd, method="revised simplex")

[[-0.01950922  0.          0.          1.        ]
 [-0.04513889  0.          0.          1.        ]
 [-0.09027778  0.          0.04513889  1.        ]
 [-0.09027778  0.02256944  0.          1.        ]
 [ 0.03009259  0.         -0.09027778  1.        ]
 [ 0.          0.         -0.06018519  1.        ]
 [ 0.          0.03009259 -0.09027778  1.        ]
 [ 0.         -0.5         1.          0.        ]
 [ 1.         -0.5         0.          0.        ]
 [ 0.         -0.02256944  0.          1.        ]
 [ 0.11284722 -0.06770833  0.          1.        ]
 [ 0.         -0.06770833  0.11284722  1.        ]
 [ 0.         -0.04513889  0.          1.        ]
 [ 0.18055556 -0.09027778  0.          1.        ]]


In [28]:
opt_1

     con: array([0.])
     fun: -0.0007336161988099899
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([ 0.00000000e+00,  9.63766789e-04,  1.04048365e-03,  7.80362739e-04,
        1.37612739e-03,  1.42727197e-03, -4.33680869e-19,  5.76267869e-03,
        4.06310510e-03,  1.14717084e-03,  6.65287444e-04,  8.57079602e-04,
        3.02795788e-03, -3.25260652e-19])
  status: 0
 success: True
       x: array([0.03760356, 0.08333333, 0.03590399, 0.00073362])

## Heather: Convex Combo of Max Entropy and Matroid Intersection 

In [33]:
c = [0, 0, 0, -1] # tau, beta, gamma, z

lam = 0
values = []
while lam <= 1:

    #lam = 1/2 #add in a for loop for this 

    p_sp_entropy = 128/6561
    p_entropy = 13/144 
    p_hs_entropy = 13/144

    p_sp_matroid = 1/36
    p_matroid = 1/18
    p_hs_matroid = 1/21 


    p_sp = (lam * p_sp_entropy) + ((1-lam) * p_sp_matroid)
    p = (lam * p_entropy) + ((1-lam) * p_matroid)
    p_hs = (lam * p_hs_entropy) + ((1-lam) * p_hs_matroid)


    ## CHARGING DEGREE
    #p_internal, p_external = min(p, p_hs), max(p, p_hs) # NOTE! in non-uniform, we can do p >= 1/9. 
    HALLS_CONSTANT = 1/2
    P_CUT_ODD_DEGREE = 1
    P_CUT_ODD_CYCLE = 1/2

    special_edge = ([-1 * p_sp, 0, 0, 1], 0) # Special edge 
    degree_degree = ([-1 * (p_hs - p * HALLS_CONSTANT*P_CUT_ODD_DEGREE), 0, 0, 1], 0) # Degree edge, all external edges are degree
    degree_k5 = ([-1 * p_hs, 0, p * HALLS_CONSTANT*P_CUT_ODD_DEGREE, 1],0) # Degree edge, all external edges are K_5
    degree_cycle = ([-1 * p_hs, p * HALLS_CONSTANT*P_CUT_ODD_CYCLE, 0, 1],0) # Degree edge, all external edges are cycle



    ## CHARGING K5
    #p_internal, p_external = p, max(p, p_hs)  # NOTE! in non-uniform, we can do p_internal >= 1/9. 
    HALLS_WEAK_CONSTANT = 2/3
    P_CUT_ODD_K5 = 1/2

    k5_degree = ([p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, 0, -1 * p, 1], 0)
    k5_k5 = ([0, 0, -1 * p + p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, 1], 0)
    k5_cycle = ([0, p*HALLS_WEAK_CONSTANT*P_CUT_ODD_K5, -1 * p, 1], 0)



    ## CHARGING CYCLE (added in one more case here)
    gamma_le_betaover2 = ([0, -1/2, 1, 0], 0) #assume cycle edges give worst case (see 7.1.2 (iv))
    tau_le_betaover2 = ([1, -1/2, 0, 0], 0) #assume cycle edges give worst case (see 7.1.2 (iv))

    cycle_i = ([0, -1 * p * 1/4, 0, 1],0) # Constraint from case 7.1.2 (i)
    cycle_ii = ([p * 5/4, -1 * p * 3/4, 0 , 1],0) # Constraint from case 7.1.2 (ii), degree case
    cycle_ii_2 = ([0, -1 * p * 3/4, p * 5/4 , 1],0) # Constraint from case 7.1.2 (ii), K5 case
    cycle_iii = ([0, -1 * p * 1/2, 0, 1],0) # Constraint from case 7.1.2 (iii)
    cycle_iv = ([2 * p, -1 * p, 0, 1],0) # Constraint from case 7.1.2 (iv)

    ### 

    constraints = [special_edge, degree_degree, degree_k5, degree_cycle, k5_degree, k5_k5, k5_cycle, gamma_le_betaover2, tau_le_betaover2, 
                  cycle_i, cycle_ii, cycle_ii_2, cycle_iii, cycle_iv]


    lhs_ineq_1 = np.array([cons[0] for cons in constraints])
    rhs_ineq_1 = np.array([cons[1] for cons in constraints])

    #print(lhs_ineq_1)

    bnd = [(0, float("inf")) for variable in c] 

    opt_1 = linprog(c=c, A_ub=lhs_ineq_1, b_ub=rhs_ineq_1, A_eq=[[0, 1, 0, 0]], b_eq=[1/12], bounds=bnd, method="revised simplex")
    
    values.append(-1 * opt_1.fun)
    lam += 0.001
print(max(values))

0.0008635287322383441
